In [4]:
from cv2 import cv2
import numpy as np
from matplotlib import pyplot as plt
import statistics
import mahotas       # Otsu thresholding
import bisect        # Key insert point
import imutils       # Crop and resize images
import pandas as pd

from __future__ import unicode_literals
import youtube_dl
import os            # Folder paths
import sys           # Exit function
import glob          # Folder searching

In [35]:
# note class
class Note: 
    def __init__(self, centroid_x, y_dot):
        self.centroid_x = centroid_x
        self.y_dot = y_dot

# just  a function for printing images
def display_img(title, img):
    cv2.imshow(title, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
      
# function for getting next note
def getNextNote(first_note):
    if "#" in first_note: 
        octave = first_note[2]
        if first_note[:2] == "A#":
            return ("C#" + octave)
        elif first_note[:2] == "C#":
            return ("D#" + octave)
        elif first_note[:2] == "D#":
            return ("F#" + octave)
        elif first_note[:2] == "F#":
            return ("G#" + octave)
        elif first_note[:2] == "G#":
            next_octave = int(octave) + 1
            return ("A#" + str(next_octave))
    
    else: 
        octave = first_note[1]
        if first_note[0] == "A":
            return ("B" + octave)
        elif first_note[0] == "B":
            return ("C" + octave)
        elif first_note[0] == "C":
            return ("D" + octave)
        elif first_note[0] == "D":
            return ("E" + octave)
        elif first_note[0] == "E":
            return ("F" + octave)
        elif first_note[0] == "F":
            return ("G" + octave)
        elif first_note[0] == "G":
            next_octave = int(octave) + 1
            return ("A" + str(next_octave))

# Function for Gaussian Blurring
def gaussianBlurring(gray_img, blur_sq, std_dev = 0):
    return cv2.GaussianBlur(gray_img, (blur_sq, blur_sq), std_dev)

# Function for Canny Edge Detection
def cannyDetection(blurred_img, th1, th2, apertureSize = 3):
    return cv2.Canny(blurred_img, th1, th2, apertureSize)

# Function to return the top and bottom of the keyboard using HoughLines
def keyboardYCoords(edged_img, rho = 1, theta = np.pi/180, threshold = None):
    
    if threshold is None:
        threshold = edged_img.shape[1]//2 # half the width
    
    lines = cv2.HoughLines(edged_img, rho, theta, threshold) 
    y_cord = [] #the y-value of the lines generated from hough transform

    #iterating through lines
    for line in lines: 
        rho_l, theta_l = line[0]
        a = np.cos(theta_l)
        b = np.sin(theta_l)
        x0 = a * rho_l
        y0 = b * rho_l
        y_cord.append(y0) #appending to list

    y_cord.sort(reverse=True)
    return y_cord[0:2]

# Function to threshold an image
def threshold(gray_img, th1 = 90, th2 = 150, thresh_type = cv2.THRESH_BINARY_INV):
    _, threshed_img = cv2.threshold(gray_img, th1, th2, thresh_type)
    return threshed_img


        
# Function for doing connected components
def connectedComponents(binarized_img, connectivity = 8, ltype = cv2.CV_32S):
    output = cv2.connectedComponentsWithStats(binarized_img, connectivity, cv2.CV_32S)
    num_labels = output[0]
    labels = output[1]
    stats = output[2]
    centroids = output[3]

    return [num_labels, labels, stats, centroids]


# Function to calculate the centroids of detected connected components
#   This should be used for the 36 black keys and 52 white keys
#   use display_result = True to display the results on the source image
def keyDetection(orig_img, num_labels, labels, stats, centroids, min_key_area = 100, display_result = False):
    final_labels = []

    output_img = orig_img.copy()

    # Loop through the detected connected components
    for i in range(1, num_labels):
        [x,y,w,h,area] = getConnectedComponentRectangle(stats[i,:])
        (cX, cY) = centroids[i]
        
        # Consider only connected that have an area greater than min_key_area pixels
        if (min_key_area < area < np.inf):
            final_labels.append([i,cX])
            
            # Display on original image
            if (display_result):
                cv2.rectangle(output_img, (x,y), (x+w, y+h), (255,0,0),1)
                cv2.circle(output_img, (int(cX), int(cY)), 4, (255,255,0), -1)
#                 componentMask = (labels == i).astype("uint8") * 255
                cv2.imshow("Output", output_img)
                cv2.waitKey(0)

    key_width = statistics.median(stats[:, cv2.CC_STAT_WIDTH])
    cv2.destroyAllWindows()
    
    #Return the centroid of the processed connected components and the median width
    return final_labels, key_width

# Get the x,y coordinates and width + height of cv2.connectedcomponents output
#    Given the i-th components of the stats matrix (an N x 5 matrix)
def getConnectedComponentRectangle(ith_stats):
        x = ith_stats[cv2.CC_STAT_LEFT]
        y = ith_stats[cv2.CC_STAT_TOP]
        w = ith_stats[cv2.CC_STAT_WIDTH]
        h = ith_stats[cv2.CC_STAT_HEIGHT]
        area = ith_stats[cv2.CC_STAT_AREA]
        
        return [x,y,w,h,area]


def displayCentroid(key_list, img):
    y = img.shape[0]*3//4
    for (note, centroid) in key_list: 
        font = cv2.FONT_HERSHEY_SIMPLEX
        line = cv2.line(img,(int(centroid),0),(int(centroid),900),(0,0,255),1)
        text_label = cv2.putText(img, note, (int(centroid), y), font, 0.5, (0,255,0), 1)
        cv2.imshow("Key Label", img)
        cv2.waitKey(0)
    cv2.destroyAllWindows()

# Given an x-coordinate, return the appropriate insertion point in the keyboard
def key_pressed(key_list, key_index):
    insertion_point = bisect.bisect_left(key_list[:,1].astype(float),key_index)
    
    #Insertion outside our index, means to insert it at the end (return the last key)
    if insertion_point >= len(key_list):
        insertion_point = len(key_list)-1
#     print(insertion_point)
#     print('You pressed the {} key.'.format(key_list[insertion_point,0]))

    note = key_list[insertion_point,0]
    index = insertion_point

    return note, index





def timeNotes(ith_centroids, y_offset, note_height, keyboard_line, keyboard_array, keys_timed_update, 
              elapsed, output_img, font = cv2.FONT_HERSHEY_SIMPLEX, font_scale = 0.5, font_color = (0,0,255)):
    # ith_centroids - the centroid of a given note (i.e. an x,y pair)
    # y_offset - if the keyboard has been offset in the y direction (this allows the detection to stay out of the top/bottom edges)
    # note_height - from connectedcomponents
    # keyboard_line - the y-coordinate that serves as the threshold/line to calculate press down/up
    # keyboard_array - the matrix of keyboard notes/octaves and x-coordinates
    # key_timed_update - our array that holds on/off button for each note in our keyboard
    # elapsed - time in seconds
    # output_img - the screen we want to display to

    lag = 2 # number of pixels about the keyboard_line for detection
    
    # Get centroid of detected note
    (cX, cY) = ith_centroids
    cY = cY + y_offset # We cropped out the first 20 pixels

    # Y-coordinate of top and bottom edges of a note (this is truncated by the "size" of the detection area)
    dist_to_edge = note_height/2 #getting the distance from centroid to bottom edge for better detection later on
    top_dot = cY-dist_to_edge
    bottom_dot = cY+dist_to_edge

#     note = Note(cX, cY+dist_to_edge) #creating note object and adding to list

    if ( (int(bottom_dot) >= int(keyboard_line - lag)) and (int(bottom_dot) <= int(keyboard_line)) ):
        note_played, index = key_pressed(keyboard_array, cX)
        keys_timed_update[index].append([elapsed])
#         print(note_played)
#         print('='*50)

    if ( (int(top_dot) >= int(keyboard_line - lag)) and (int(top_dot) <= int(keyboard_line)) ):
        note_played, index = key_pressed(keyboard_array, cX)
        keys_timed_update[index].append([elapsed])

    note_played, _ = key_pressed(keyboard_array, cX)
    cv2.line(output_img, (0, int(keyboard_line)), (600, int(keyboard_line)), (0,0,255), 2)
    cv2.rectangle(output_img, (x,y), (x+w, y+h), font_color,1)
    cv2.circle(output_img, (int(cX), int(bottom_dot)), 1, (0,122,255), 3)
    cv2.circle(output_img, (int(cX), int(top_dot)), 1, (0,122,255), 3)
    cv2.putText(output_img, note_played, (int(cX), int(cY+dist_to_edge)), font, font_scale, font_color, 1)
    
    return keys_timed_update

- Ask user for YouTube link
- Download (?) video (or atleast a temporary copy)
- Using the first few frames, detect the keyboard
    - If 72 keys are not detected, try next few frames and repeat
    - If they can't be detected at all, end program
- Label black and white keys
- Run algorithm, get array with start/end times for each note
- Convert to .midi
- Convert to sheet music

In [6]:
class Keyboard: 
    def __init__(self, youtubeURL):
        self.youtubeURL = youtubeURL
        
    # For downloading YouTube videos
    def my_hook(self, d):
        if d['status'] == 'finished':
            print('Download complete.')
        elif d['status'] == 'error':
            print('Error in downloading file - exiting program!')
            sys.exit()
            
    # Remove previously downloaded file
    def clear_previous(self, path = "./videos/video_to_process*"):
        vtp = glob.glob(path)
        if vtp:
            camera = cv2.VideoCapture(vtp[0])
            camera.release()
            os.remove(vtp[0])

    # Function to download a YouTube video
    def downloadYouTube(self, path = './videos/video_to_process.%(ext)s', quiet = True):
    
        self.clear_previous()
    
        ydl_opts = {'outtmpl': path,
                   'quiet': quiet,
                   'progress_hooks': [self.my_hook]}
        try:
            with youtube_dl.YoutubeDL(ydl_opts) as ydl:
                print('Downloading video...')
                ydl.download([self.youtubeURL])
        except youtube_dl.utils.DownloadError:
            print('Exiting program!')
            
    # Use the file called video_to_process and detect our 88 keys
    def detect_keys(self, resize_width, bl_blur_sq, bl_canny_th1, bl_canny_th2, bl_thresh1, bl_thresh2,
                wh_blur_sq, path = "./videos/video_to_process*",num_bl_keys = 36, num_wh_keys = 52):
        vtp = glob.glob(path)
        if vtp:
            camera = cv2.VideoCapture(vtp[0])
            # Could not access the file
            if not camera.isOpened():
                print('Error - video file could not be read!')
                sys.exit()
            # Process video
            else:
                print('Detecting keys...')
                while True:
                    #grabbed is a boolean than tells us if there is a valid frame
                    (grabbed, frame) = camera.read()
                    if not grabbed:
                        break

                    frame = imutils.resize(frame,width = resize_width)

                    # Get the bottom-half of the frame (where the keyboard lies) and process from here
#                     keys = frame[frame.shape[0]//2:,:]
#                     gray_keys = cv2.cvtColor(keys, cv2.COLOR_BGR2GRAY)
                    keys = frame
                    gray_keys = cv2.cvtColor(keys, cv2.COLOR_BGR2GRAY)

                    # Process
                    blurred = gaussianBlurring(gray_keys, blur_sq = bl_blur_sq)
                    edges = cannyDetection(blurred, th1 = bl_canny_th1, th2 = bl_canny_th2)

                    # Crop
                    crop_coordinates  = keyboardYCoords(edges)
                    keyboard.y_coords = crop_coordinates
                    cropped_keys      = keys[int(crop_coordinates[1])+20:int(crop_coordinates[0])]
                    cropped_gray_keys = gray_keys[int(crop_coordinates[1])+20:int(crop_coordinates[0])]

                    # Labels keys
                    thresh_keys_bl = threshold(cropped_gray_keys, th1 = bl_thresh1, th2 = bl_thresh2)

                    # Black keys
                    [num_labels_bl, labels_bl, stats_bl, centroids_bl] = connectedComponents(binarized_img = thresh_keys_bl)
                    final_labels_bl, key_width_bl = keyDetection(cropped_keys, num_labels_bl, labels_bl, stats_bl, centroids_bl)
                    if len(final_labels_bl) == num_bl_keys: 
                        first_note = "A#0"
                        for i in range(num_bl_keys):
                            final_labels_bl[i][0] = first_note
                            first_note = getNextNote(first_note)

                        final_labels_bl = sorted(final_labels_bl, key=lambda x: x[1])

                    # White keys
                    blurred_w = gaussianBlurring(cropped_gray_keys, blur_sq = wh_blur_sq)
                    T = mahotas.thresholding.otsu(blurred_w)*1.3
                    thresh_keys_w = cropped_gray_keys.copy()
                    thresh_keys_w[thresh_keys_w>T] = 255
                    thresh_keys_w[thresh_keys_w<T] = 0
                    [num_labels_w, labels_w, stats_w, centroids_w] = connectedComponents(binarized_img = thresh_keys_w)
                    final_labels_w, key_width_w = keyDetection(cropped_keys, num_labels_w, labels_w, stats_w, centroids_w)

                    if len(final_labels_w) == num_wh_keys: 
                        first_note = "A0"
                        for j in range(num_wh_keys):
                            final_labels_w[j][0] = first_note
                            first_note = getNextNote(first_note)

                        final_labels_w = sorted(final_labels_w, key=lambda x: x[1])

                    # Determine if they sum to 88 keys (36 black, 52 white) - if not, try next frame
                    if len(final_labels_bl) == num_bl_keys and len(final_labels_w) == num_wh_keys:
                        self.black_keys = final_labels_bl
                        self.white_keys = final_labels_w
                        self.black_width = key_width_bl
                        self.white_width = key_width_w
                        self.keyboard_img = frame
                        camera.release()         # Release cv2 camera object
                        cv2.destroyAllWindows()  # Destroy any cv2 windows
                        print('Key detection complete.')
                        break

                #     #Show the frame + drawn rectangle
                #     cv2.imshow("Face", thresh_keys_bl)

    #                 #Can break early by pressing "q"
    #                 if cv2.waitKey(1) & 0xFF == ord("q"):
    #                     break

            # If at this point we've looped through everything and we don't have 72 keys
            camera.release()
            cv2.destroyAllWindows()

            if len(final_labels_bl) != 36 and len(final_labels_w) != 52:
                print('Error in processing file - exiting program!')
                sys.exit()
                
                
                
        # Glob did not find a valid file
        else:
            print('Error - video path could not be accessed!')
            sys.exit()
            
            
            
    # Assign end-of-range for each key
    def key_ranges(self):

        # Array
        full_key_list = self.black_keys + self.white_keys
        full_key_list = sorted(full_key_list, key=lambda x: x[1].astype(float))
        full_key_list = np.array(full_key_list)

        # Empty list
        tmp_list = np.empty([len(full_key_list), 2], dtype='object')

        # Loop through and assign end-of-range for each key
        # CASE: white key adjacent to black key: end of the white key range is the adjacent black key's centroid - black/2
        #       black key:                       end of the black key range is the black key's centroid + black/2
        #       white key adjacent to white key: end of the white key range is the half-way point between the adjacent centroids
        for i in range(0,len(full_key_list)-1):
            if len(full_key_list[i,0])==1 and len(full_key_list[i+1,0])>1: # White adjacent to black
                tmp_list[i,1] = full_key_list[i+1,1].astype(float) - self.black_width/2
            elif len(full_key_list[i,0])>1: # Black key
                tmp_list[i,1] = full_key_list[i,1].astype(float) + self.black_width/2
            else: # White key adjacent to white key
                tmp_list[i,1] = (full_key_list[i,1].astype(float)+ full_key_list[i+1,1].astype(float))/2

            # No change to the actual note (only the distances, above) for the first key
            tmp_list[i,0] = full_key_list[i,0]

        #For the last key, just take it to infinity
        tmp_list[-1,1] = np.inf
        tmp_list[-1,0] = full_key_list[-1,0]

        full_key_list = tmp_list

        self.keyboard_array = full_key_list
            
    
    # Return the centroid of the black/white keys alongside the median width of each
    def getKeys(self):
        return [self.black_keys, self.white_keys, self.black_width, self.white_width]
    
    # Return the frame that 88 keys were successfully identified from
    def getFrame(self):
        return self.keyboard_img
    
    def getFullKeyList(self):
        return self.keyboard_array
    
    def getKeyboardYCoords(self):
        return keyboard.y_coords

In [7]:
#Download a YouTube video and process it to determine where the centroid of the black and white keys are
keyboard = Keyboard('https://www.youtube.com/watch?v=skFugVOqBM4')
# keyboard.downloadYouTube()
keyboard.detect_keys(resize_width = 600, bl_blur_sq = 5, bl_canny_th1 = 200, bl_canny_th2 = 200, 
                    bl_thresh1 = 90, bl_thresh2 = 150, wh_blur_sq = 7)
keyboard.key_ranges()

[black, white, black_width, white_width] = keyboard.getKeys()     # Array of keys
key_img = keyboard.getFrame()                                     # Image of video used to detect our 88 keys
keyboard_array = keyboard.getFullKeyList()                        # Combined array of keys
keyboard_y_coords = keyboard.getKeyboardYCoords()                 # y[0] is the bottom of the keyboard, y[1] is the top

Detecting keys...
Key detection complete.


In [165]:
# keyboard_y_coords

[331.0, 268.0]

In [32]:
# # plt.imshow(key_img)
# displayCentroid(white, key_img.copy())

In [33]:
# keyboard_array
#Find the appropriate index for any given centroid via:
# note, index = key_pressed(full_key_list, 999)
# print(index)

array([['A0', 8.248157248157248],
       ['A#0', 16.363636363636363],
       ['B0', 21.35514018691589],
       ['C0', 30.73469387755102],
       ['C#0', 36.515625],
       ['D0', 43.51020408163265],
       ['D#0', 50.484375],
       ['E0', 55.393081761006286],
       ['F0', 65.71260997067449],
       ['F#0', 71.515625],
       ['G0', 77.83985765124555],
       ['G#0', 83.50393700787401],
       ['A1', 89.6822429906542],
       ['A#1', 96.484375],
       ['B1', 101.73578595317726],
       ['C1', 111.7280701754386],
       ['C#1', 117.50393700787401],
       ['D1', 123.90615835777126],
       ['D#1', 131.48818897637796],
       ['E1', 136.371875],
       ['F1', 146.5943396226415],
       ['F#1', 151.515625],
       ['G1', 158.328125],
       ['G#1', 164.5],
       ['A2', 170.36789297658862],
       ['A#2', 177.48818897637796],
       ['B2', 182.371875],
       ['C2', 192.628125],
       ['C#2', 197.515625],
       ['D2', 204.51020408163265],
       ['D#2', 211.48818897637796],
       ['E

In [ ]:
# Process and output...

In [11]:
# Need to delete the processed video
# keyboard.clear_previous()

# IGNORE BELOW THIS LINE ##########################
___________________________________________________________________________________________________________________________

# 4. Testing with Notes

In [ ]:
# #sample piano image from  youtube
# img_notes = cv2.imread("images/IMG_very_close_notes.png")

# #converting to gray
# gray_notes = cv2.cvtColor(img_notes, cv2.COLOR_BGR2GRAY)
# img_notes_rgb = cv2.cvtColor(img_notes, cv2.COLOR_BGR2RGB)

# plt.imshow(img_notes_rgb)
# # plt.axis('off')
# plt.show()

*y_cord[1]* is the position of the top of the keyboard. We calculated this using **half** the original image, so we need to add this back in.

# I added it back paps =3

In [ ]:
# #Cropped
# top_keys_index = y_cord[1] + img_notes.shape[0]//2
# crop_img_notes = img_notes[20:(int(top_keys_index)-30)] #Crop the top 20 pixels and bottom 30

# plt.imshow(crop_img_notes)

Identify notes.

In [ ]:
# crop_img_notes_gray = cv2.cvtColor(crop_img_notes, cv2.COLOR_BGR2GRAY)
# # # k = 3
# # # blurred = cv2.GaussianBlur(crop_img_notes_gray, (k,k), 0)

# # #Using standard threshold to create contrast between white/black keys
# # _, th_notes = cv2.threshold(blurred, 90, 150, cv2.THRESH_BINARY)

# _, th_notes = cv2.threshold(crop_img_notes_gray, 90, 150, cv2.THRESH_BINARY)

# plt.figure(figsize=(10,10))
# plt.imshow(th_notes, cmap = "gray")
# plt.show()

In [ ]:
# #####################################################################################
# ####### using connected component detection algorithm to separate all the black notes
# connectivity = 1
# output = cv2.connectedComponentsWithStats(th_notes, connectivity, cv2.CV_32S)
# num_labels = output[0]
# labels = output[1]
# stats = output[2]
# centroids = output[3]

# final_labels = []
# note_list = [] #creating a list of all the relavent notes. 


# output = img_notes_rgb.copy()
# font = cv2.FONT_HERSHEY_SIMPLEX

# #For loop only used for displaying 
# for i in range(1, num_labels):
#     x = stats[i, cv2.CC_STAT_LEFT]
#     y = stats[i, cv2.CC_STAT_TOP] + 20 # We cropped out the first 20 pixels
#     w = stats[i, cv2.CC_STAT_WIDTH]
#     h = stats[i, cv2.CC_STAT_HEIGHT]
#     area = stats[i, cv2.CC_STAT_AREA]
#     (cX, cY) = centroids[i]
#     cY = cY + 20 # We cropped out the first 20 pixels
#     if (20 < area < np.inf): #filtering out relavent detections (the ones big enough to be black keys)
#         final_labels.append(i)
#         cv2.rectangle(output, (x,y), (x+w, y+h), (255,0,0),1)
#         dist_to_edge = h/2 #getting the distance from centroid to bottom edge for better detection later on
#         cv2.circle(output, (int(cX), int(cY+dist_to_edge)), 1, (0,122,255), 3)
#         componentMask = (labels == i).astype("uint8") * 255
        
#         note = Note(cX, cY+dist_to_edge) #creating note object and adding to list
#         note_list.append(note)
        
#         note_played, _ = key_pressed(full_key_list, note.centroid_x)
#         cv2.putText(output, note_played, (int(note.centroid_x), int(note.y_dot)), font, 0.5, (0,255,0), 1)

# #         display_img("Output", output)
# #         display_img("Connected Component", componentMask)
# #         cv2.waitKey(0)

# # print(final_labels)
# # cv2.destroyAllWindows()
# plt.figure(figsize=(10,10))
# plt.imshow(output)
# plt.show()

In [36]:
camera = cv2.VideoCapture('videos/Moonlight Sonata 1st Movement  Opus 27 No 2_Trim.mp4')

frames = camera.get(cv2.CAP_PROP_FRAME_COUNT)
fps = camera.get(cv2.CAP_PROP_FPS)
seconds_per_frame = fps/frames



counter = 0

notes_pressed = []

keys_timed = []
for x in keyboard_array:
    keys_timed.append([x[0]])


keys_timed_update = []
for x in keyboard_array:
    keys_timed_update.append([x[0]])



while (camera.isOpened()):    
    #grabbed is a boolean than tells us if there is a valid frame
    (grabbed, frame) = camera.read()

    frame_number = camera.get(cv2.CAP_PROP_POS_FRAMES)
    elapsed = frame_number/fps

    if not grabbed:
        break

    counter += seconds_per_frame        
         
    #!!! - width
    frame = imutils.resize(frame,width = 600) #resize or else it won't work
    
    #!!! - keyboard_y_coords//40//20
    crop_frame = frame[20:int(keyboard_y_coords[1])-40] #Crop the top 20 pixels and bottom 50
    
    # threshold the cropped and grayed image
    crop_frame_gray = cv2.cvtColor(crop_frame, cv2.COLOR_BGR2GRAY)
    th_crop_frame = threshold(crop_frame_gray, thresh_type = cv2.THRESH_BINARY)

    
    # We have a thresholded image for use to use ConnectedComponents on
    #####################################################################################
    ####### using connected component detection algorithm to separate all the black notes    
    [num_labels, labels, stats, centroids] = connectedComponents(th_crop_frame)
    
    # List of indices that we may have to remove - combined white/black key needs to be split
    indices_to_pop = []

    output_img = frame.copy()
    font = cv2.FONT_HERSHEY_SIMPLEX

    
    i=1
    #Loop through all the connected components
    while i < len(stats):

        # Width of i-th connected component
        curr_connected_comp = stats[i, cv2.CC_STAT_WIDTH]

        # !!! - Determine if the WIDTH is much bigger than the width of a white key and less than 3x? (So we don't get extraneous video text, etc.)
        if curr_connected_comp > white_width*1.25 and curr_connected_comp < white_width*3:
            
            #Threshold just the large component of interest
            componentMask = (labels == i).astype("uint8") * 255
            threshMask = cv2.bitwise_and(crop_frame_gray, crop_frame_gray, mask = componentMask) #Replace this with video frame

            # Histogram segregation of black/white key
            # Grayscale has one channel so we use [0]
                #Possible values range from 0 to 256
            bin_scaler = 4
            hist = cv2.calcHist([threshMask], [0], None, [256/bin_scaler], [1, 256])

            #Use a Histogram to compute the dominant non-black (i.e. not the background) colour. Use ~90% of this to threshold the image.
            T = hist.argmax() * bin_scaler * .95
            white_notes = threshMask.copy()
            white_notes[white_notes>T] = 255
            white_notes[white_notes<T] = 0

            #Detect the first set of WHITE keys
            [num_labels_wh, labels_wh, stats_wh, centroids_wh] = connectedComponents(white_notes)

            #Loop through components and determine which ones may be keys
            for j in range(1, num_labels_wh):
                area = stats_wh[j, cv2.CC_STAT_AREA]
                # !!! - min white pixel area
                if (20 < area < np.inf): #filtering out relavent detections (the ones big enough to be keys)

                    if j > 1:
                        ## We've added another label
                        num_labels +=1 
                        i +=1

                    #Within labels, we have a matrix that is the same size of the image that holds our split component
                    #First, cut out the original "fat" label - i.e. two or more keys are coupled together
                    coupled_keys_mask = labels != i
                    labels = labels * coupled_keys_mask

                    #Next, increment each label above the cut one up to accomodate the new label
                    higher_mask = labels > i
                    labels = labels + higher_mask

                    #Then append our segregated key
                    new_mask = labels_wh == j
                    new_labels = labels_wh * new_mask
                    new_labels = i * new_labels
                    labels = labels + new_labels

                    ##Remove the original index for the stats and then add the new one
                    if i < len(stats):
                        stats = np.delete(stats,i,0)
                        stats = np.insert(stats,i,stats_wh[j],0)
                    elif j == 1:
                        stats = stats[:-1,:]
                        stats = np.concatenate((stats,stats_wh[j][None,:]),0)
                    else:
                        stats = np.concatenate((stats,stats_wh[j][None,:]),0)

                    ##Remove the original index for the centroids and then add the new one
                    if i < len(centroids):
                        centroids = np.delete(centroids,i,0)
                        centroids = np.insert(centroids,i,centroids_wh[j],0)
                    elif j==1:
                        centroids = centroids[:-1,:]
                        centroids = np.concatenate((centroids,centroids_wh[j][None,:]),0)  
                    else:
                        centroids = np.concatenate((centroids,centroids_wh[j][None,:]),0)  
                        

                        
                    #Plot immediately so indexing doesn't get messed up
                    # !!! - +20 pixels
                    [x,y,w,h,area] = getConnectedComponentRectangle(stats[i,:])
                    y += 20 # We cropped out the first 20 pixels
                    
                    if w >= black_width * 0.5:
                        keys_timed_update = timeNotes(centroids[i], 20, h, keyboard_y_coords[1]-50, keyboard_array, 
                                                      keys_timed_update, elapsed, output_img,
                                                      font = font, font_scale = 0.5, font_color = (128,0,128))


                #Detect the next set of keys
                black_tmp = threshMask.copy()
                black_tmp[black_tmp>T] = 0        
                k = 5
                blurred_black_notes = cv2.GaussianBlur(black_tmp, (k,k), 0)

                #Using standard threshold to create contrast between white/black keys
                black_notes = threshold(blurred_black_notes, thresh_type = cv2.THRESH_BINARY)

                #Detect the second set of keys
                [num_labels_bl, labels_bl, stats_bl, centroids_bl] = connectedComponents(black_notes)
                

                #Loop through components and determine which ones may be keys
                for k in range(1, num_labels_bl):
                    area = stats_bl[k, cv2.CC_STAT_AREA]
                    
                    # !!! - min area
                    if (20 < area < np.inf): #filtering out relavent detections (the ones big enough to be keys)

                        if k > 1:
                            ## We've added another label
                            num_labels +=1 
                            i+=1

                        #For the second set of keys WE DON'T NEED TO CUT anything
        #                 coupled_keys_mask = labels != i
        #                 labels = labels * coupled_keys_mask

                        #Next, increment each label above the cut one up to accomodate the new label
                        higher_mask = labels > i + 1
                        labels = labels + higher_mask

                        #Then append our segregated key
                        new_mask = labels_bl == k
                        new_labels = labels_bl * new_mask
                        new_labels = (i + 1) * new_labels
                        labels = labels + new_labels

                        ##Add
                        if i < len(stats):
                            stats = np.insert(stats,(i+1),stats_bl[k],0)                       
                        else:
                            stats = np.concatenate((stats,stats_bl[k][None,:]),0)
                        

                        ##Add
                        if i < len(centroids):
                            centroids = np.insert(centroids,(i+1),centroids_bl[k],0)
                        else:
                            centroids = np.concatenate((centroids,centroids_bl[k][None,:]),0)

                        #Plot immediately so indexing doesn't get messed up
                        # !!! - +20 pixels
                        [x,y,w,h,area] = getConnectedComponentRectangle(stats[i+1,:])
                        y += 20 # We cropped out the first 20 pixels

                        
                        # !!! - minimum black width
                        if w >= black_width * 0.5:
                            keys_timed_update = timeNotes(centroids[i+1], 20, h, keyboard_y_coords[1]-50, keyboard_array, 
                                                      keys_timed_update, elapsed, output_img,
                                                      font = font, font_scale = 0.5, font_color = (0,0,255))

        else:
            # !!! - +20 pixels
            [x,y,w,h,area] = getConnectedComponentRectangle(stats[i,:])
            y += 20 # We cropped out the first 20 pixels
            
            
            if (20 < area < np.inf) and w >= black_width * 0.5: #filtering out relavent detections (the ones big enough to be black keys)
                keys_timed_update = timeNotes(centroids[i], 20, h, keyboard_y_coords[1]-50, keyboard_array, 
                                                      keys_timed_update, elapsed, output_img,
                                                      font = font, font_scale = 0.33, font_color = (255,255,255))

        i+=1
    
    
    #Show the frame + drawn rectangle
    cv2.imshow("Video", output_img)

    #Can break early by pressing "q"
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
        
    if cv2.waitKey(1) & 0xFF == ord("p"):
        cv2.waitKey(-1) #wait until any key is pressed

# print(keys_timed_update)    
camera.release()
cv2.destroyAllWindows()

In [37]:
keys_timed_update

[['A0'],
 ['A#0'],
 ['B0'],
 ['C0'],
 ['C#0'],
 ['D0'],
 ['D#0'],
 ['E0'],
 ['F0'],
 ['F#0', [0.6333333333333333], [0.6666666666666666], [3.2333333333333334]],
 ['G0'],
 ['G#0', [15.5]],
 ['A1', [13.066666666666666], [13.1], [15.466666666666667]],
 ['A#1'],
 ['B1'],
 ['C1', [10.533333333333333], [10.566666666666666], [11.766666666666667]],
 ['C#1',
  [0.6333333333333333],
  [8.133333333333333],
  [9.333333333333334],
  [11.766666666666667],
  [13.033333333333333]],
 ['D1'],
 ['D#1',
  [3.2666666666666666],
  [6.866666666666666],
  [9.366666666666667],
  [10.533333333333333]],
 ['E1', [6.9], [8.1]],
 ['F1'],
 ['F#1', [0.6666666666666666], [3.2333333333333334]],
 ['G1'],
 ['G#1', [15.5]],
 ['A2', [13.066666666666666], [13.1], [15.466666666666667]],
 ['A#2'],
 ['B2'],
 ['C2', [10.533333333333333], [10.566666666666666], [11.766666666666667]],
 ['C#2',
  [0.6333333333333333],
  [8.133333333333333],
  [9.333333333333334],
  [11.8],
  [13.033333333333333]],
 ['D2'],
 ['D#2',
  [3.266666666666

In [38]:
new_list = []

for i in range (len(keys_timed_update)):
    temp = []
    temp.append(keys_timed_update[i][0])
    if (len(keys_timed_update[i]) > 1):
        for j in range(1,len(keys_timed_update[i])):
            temp.append(keys_timed_update[i][j][0])

    new_list.append(temp)



df = pd.DataFrame(new_list)
df.to_csv('notes_info.csv', index=False, header=False)